# Projeto 2 - Ciência dos Dados

Nome: Arthur Carvalho

Nome: Guilherme Lunetta

___
# Classificador automático de sentimento


## Preparando o ambiente no jupyter:

In [1]:
%%capture

#Instalando o tweepy
!pip install tweepy

In [2]:
import tweepy
import math
import os.path
import pandas as pd
import json
from random import shuffle
import emoji
import emoji_list

ModuleNotFoundError: No module named 'emoji'

___
## Autenticando no  Twitter

* Conta: @tuca1209

In [ ]:
#Dados de autenticação do twitter:

#Coloque aqui o identificador da conta no twitter: Arthur FC

#leitura do arquivo no formato JSON
with open('auth.pass') as fp:    
    data = json.load(fp)

#Configurando a biblioteca. Não modificar
auth = tweepy.OAuthHandler(data['consumer_key'], data['consumer_secret'])
auth.set_access_token(data['access_token'], data['access_token_secret'])

___
## Etapas do projeto:

### Escolha de um produto e coleta das mensagens


In [ ]:
#Produto escolhido:
produto = 'Trump'

#Quantidade mínima de mensagens capturadas:
n = 500
#Quantidade mínima de mensagens para a base de treinamento:
t = 300

#Filtro de língua, escolha uma na tabela ISO 639-1.
lang = 'pt'

Capturando os dados do twitter:

In [ ]:
#Cria um objeto para a captura
api = tweepy.API(auth)

#Inicia a captura, para mais detalhes: ver a documentação do tweepy
i = 1
msgs = []


for msg in tweepy.Cursor(api.search, q=produto, lang=lang, tweet_mode="extended").items():
    if msg.full_text.lower()[0] != 'r' and msg.full_text.lower()[1] != 't':
        msgs.append(msg.full_text.lower())
        i += 1
        if i > n:
            break

            
#Embaralhando as mensagens para reduzir um possível viés
shuffle(msgs)

Salvando os dados em uma planilha Excel:

In [ ]:
#Verifica se o arquivo não existe para não substituir um conjunto pronto
if not os.path.isfile('./{0}.xlsx'.format(produto)):
    
    #Abre o arquivo para escrita
    writer = pd.ExcelWriter('{0}.xlsx'.format(produto))

    #divide o conjunto de mensagens em duas planilhas
    dft = pd.DataFrame({'Treinamento' : pd.Series(msgs[:t])})
    dft.to_excel(excel_writer = writer, sheet_name = 'Treinamento', index = False)

    dfc = pd.DataFrame({'Teste' : pd.Series(msgs[t:])})
    dfc.to_excel(excel_writer = writer, sheet_name = 'Teste', index = False)

    #fecha o arquivo
    writer.save()

___
### Classificando as mensagens na coragem

Esta etapa é manual. Faça a mesma pelo Excel.

___
### Montando o Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

In [3]:
def limpar_caracteres(tabela, titulo):
    a = tabela[titulo]
    a = a.str.lower()

    itens = ['.', ':', ';', '"', "'", '?', '(', ')', '[',']',',', '\n', '\t']
    i2 = ['?', '!']
    emojis = emoji_list.all_emoji
    for emoji in emojis:
        try:
            a = a.str.replace(emoji," {} ".format(emoji) )
        except:
            pass
    for e in itens:
        a = a.str.replace(e,'')
    for e in i2:
        a = a.str.replace(e,' {} '.format(e))

    a = a.str.replace('é','e')
    a = a.str.replace('ê','e')
    a = a.str.replace('á','a')
    a = a.str.replace('ã','a')
    a = a.str.replace('ô','o')
    a = a.str.replace('ó','o')
    a = a.str.replace('ú','u')
    a = a.str.replace('ç','c')
    a = a.str.replace('í','i')
    a = a.str.replace('@',' @')
        
    tabela[titulo] = a

    return tabela

In [5]:
def split_tweet(tabela, titulo):
    a = tabela[titulo].str.split(' ')

    nova = pd.DataFrame()
    nova['palavras'] = []

    for lista in a:
        for e in ['#','' , ' ', 'rt']:
            while lista.count(e) != 0:
                lista.remove(e)

        for palavra in lista:
            if '@' in palavra and palavra != '@realDonaldTrump':
                lista.remove(palavra) 
            if palavra[:4] == 'http':
                lista.remove(palavra)
        nova = nova.append({'palavras': lista}, ignore_index=True)
                
    return nova

In [7]:
def tabela_palavras(tabela):
    a = tabela['palavras']
    prob = pd.DataFrame()
    prob['palavras'] = []
    for lista in a:
        for e in lista:
            prob = prob.append({'palavras': e}, ignore_index=True)
    
    return prob['palavras']

___
### Verificando a performance

Agora você deve testar o seu classificador com a base de Testes.

___
### Concluindo

## Aperfeiçoamento:

Os trabalhos vão evoluir em conceito dependendo da quantidade de itens avançados:

* Limpar: \n, :, ", ', (, ), etc SEM remover emojis
* Corrigir separação de espaços entre palavras e emojis ou emojis e emojis
* Propor outras limpezas e transformações que não afetem a qualidade da informação ou classificação
* Criar categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante (3 categorias: C, mais categorias conta para B)
* Explicar por que não posso usar o próprio classificador para gerar mais amostras de treinamento
* Propor diferentes cenários para Naïve Bayes fora do contexto do projeto
* Sugerir e explicar melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* Montar um dashboard que periodicamente realiza análise de sentimento e visualiza estes dados

# Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**